In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import re
import pandas as pd
import numpy as np

In [2]:
kprf = 'КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ'
ik_link ='http://www.krasnodar.vybory.izbirkom.ru/region/krasnodar?action=ik&numik='

# массивы для хранения результатов по участкам, в которых есть представитель КПРФ и где такого нет
kprf_per = []
no_kprf_per = []

Не получилось спарсить данные по результатам на избирательных участках, поэтому сохранила их в xls файлы с порядковыми номерами по округам. Для каждого округа в файле нахожу список участков, смотрю на сайте состав избирательной комиссии по этому участку. Если представителей КПРФ нет, сохраняю номер участка в массив. Также записываю результаты по этому участку в один из массивов, в зависимости от этого. Вывожу список таких участков по округам в консоль

In [3]:
for i in range(1, 40):
    file_name = 'report (' + str(i) + ').xls'
    xl = pd.ExcelFile(file_name)
    df1 = xl.parse('Лист1')
    uik_nums = np.array([int(j.split('№')[1]) for j in df1.loc[6].values[3:]]) # номера всех участков этого округа
    uik_per = np.array([float(i.split('%')[0]) for i in df1.loc[29].values[3:]]) # результаты по участкам
    no_kprf_uik = []
    
    for uik in uik_nums:
        page_link = ik_link + str(uik)
        page_response = requests.get(page_link, timeout=15, headers={'User-Agent': UserAgent().chrome})
        page_content = BeautifulSoup(page_response.content, "html.parser")
        idx = list(uik_nums).index(uik)
        if not(len(page_content.find_all('td', string=re.compile(kprf)))):
            no_kprf_uik.append(uik)            
            no_kprf_per.append(uik_per[idx])
        else:
            kprf_per.append(uik_per[idx])
            
            
    print('ОИО №' + str(i) + ': ' + ', '.join(map(str, no_kprf_uik)))       

ОИО №1: 2004, 2007, 2094, 2096
ОИО №2: 2018, 2099
ОИО №3: 2021, 2034
ОИО №4: 2026
ОИО №5: 
ОИО №6: 
ОИО №7: 
ОИО №8: 2101
ОИО №9: 
ОИО №10: 
ОИО №11: 
ОИО №12: 
ОИО №13: 
ОИО №14: 
ОИО №15: 2126
ОИО №16: 2180
ОИО №17: 2174
ОИО №18: 2264
ОИО №19: 
ОИО №20: 
ОИО №21: 
ОИО №22: 2239, 2266
ОИО №23: 2299
ОИО №24: 2263
ОИО №25: 
ОИО №26: 
ОИО №27: 6011, 6012, 6013, 6056, 6057, 6058
ОИО №28: 6049, 6055, 6099
ОИО №29: 6036, 6037, 6054, 6065, 6066
ОИО №30: 6031, 6052, 6060
ОИО №31: 6034, 6053, 6097, 6098
ОИО №32: 6021
ОИО №33: 6063, 6064
ОИО №34: 2393
ОИО №35: 2394, 2397, 2398, 2399
ОИО №36: 
ОИО №37: 
ОИО №38: 2395, 2396
ОИО №39: 


Считаю среднее по получившимся массивам и дельту между средними. Вывожу в консоль

In [12]:
kprf_mean = round(np.array(kprf_per).mean(), 2)
no_kprf_mean = round(np.array(no_kprf_per).mean(), 2)
print(', '.join(map(str, [kprf_mean, no_kprf_mean, abs(kprf_mean - no_kprf_mean)])))

24.1, 17.02, 7.080000000000002
